In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 39.8 MB/s eta 0:00:00


In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import numpy as np
import tensorflow as tf

from typing import Dict, Text
import pprint

### Read data

In [3]:
# ratings data
rating = tfds.load('movielens/100k-ratings', split='train')
# features of all the movies
movies = tfds.load('movielens/100k-movies', split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete84TS5M/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [6]:
for x in movies.take(10).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}
{'movie_genres': array([4, 7]),
 'movie_id': b'1457',
 'movie_title': b'Love Is All There Is (1996)'}
{'movie_genres': array([1, 3]),
 'movie_id': b'500',
 'movie_title': b'Fly Away Home (1996)'}
{'movie_genres': array([0]),
 'movie_id': b'838',
 'movie_title': b'In the Line of Duty 2 (1987)'}
{'movie_genres': array([7]),
 'movie_id': b'1648',
 'movie_title': b'Niagara, Niagara (1997)'}
{'movie_genres': array([5]),
 'movie_id': b'547',
 'movie_title': b"Young Poisoner's Handbook, The (1995)"}
{'movie_genres': array([7]),
 'movie_id': b'387',
 'movie_title': b'Age of Innocence, The (1993)'}
{'movie_genres': array([7]),
 'movie_id': b'1495',
 'movie_title': b'Flirt (1995)'}
{'movie_genres': array([7]), 'movie_id': b'817', 'movie_title': b'Frisk (1995)'}
{'movie_genres': array([17]), 'movie_id': b'267', 'movie_title': b'unknown'}


In [8]:
for x in rating.take(10).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'bucketized_user_age': 25.0,
 'movie_genres': array([ 4, 14]),
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'raw_user_age': 32.0,
 'timestamp': 875654590,
 'user_gender': True,
 'user_id': b'92',
 'user_occupation_label': 5,
 'user_occupation_text': b'entertainment',
 'user_rating': 2.0,
 'user_zip_code': b'80525'}
{'bucketized_user_age': 18.0,
 'movie_genres': array([4]),
 'movie_id': b'412',
 'movie_title': b'Very Brady Sequel, A (1996)',
 'raw_user_age': 24.0,
 'timestamp': 882075110,
 'user_gender': True,
 'user_id': b'301',
 'user_occupation_label': 17,
 'user_occupation_text': b'student',
 'user_rating': 4.0,
 'user_zip_

### Limiting the features

In [9]:
# limiting the features
rating = rating.map(lambda x:{'movie_title':x['movie_title'],'user_id':x['user_id']})
movies = movies.map(lambda x: x['movie_title'])

In [10]:
for x in movies.take(10).as_numpy_iterator():
  pprint.pprint(x)

b'You So Crazy (1994)'
b'Love Is All There Is (1996)'
b'Fly Away Home (1996)'
b'In the Line of Duty 2 (1987)'
b'Niagara, Niagara (1997)'
b"Young Poisoner's Handbook, The (1995)"
b'Age of Innocence, The (1993)'
b'Flirt (1995)'
b'Frisk (1995)'
b'unknown'


In [11]:
for x in rating.take(10).as_numpy_iterator():
  pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}
{'movie_title': b'Strictly Ballroom (1992)', 'user_id': b'92'}
{'movie_title': b'Very Brady Sequel, A (1996)', 'user_id': b'301'}
{'movie_title': b'Pulp Fiction (1994)', 'user_id': b'60'}
{'movie_title': b'Scream 2 (1997)', 'user_id': b'197'}
{'movie_title': b'Crash (1996)', 'user_id': b'601'}
{'movie_title': b'Aladdin (1992)', 'user_id': b'710'}
{'movie_title': b'True Romance (1993)', 'user_id': b'833'}
{'movie_title': b'Bob Roberts (1992)', 'user_id': b'916'}
{'movie_title': b'Starship Troopers (1997)', 'user_id': b'940'}


### Embeddings

In [12]:
user_id_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_id_vocabulary.adapt(rating.map(lambda x: x['user_id']))

movies_title_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movies_title_vocabulary.adapt(movies)

### Define TFRS Model

In [13]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

### Seprate the models

In [14]:
users_model = tf.keras.Sequential([user_id_vocabulary,
                                   tf.keras.layers.Embedding(user_id_vocabulary.vocab_size(),64)])
movie_model = tf.keras.Sequential([movies_title_vocabulary,
                                   tf.keras.layers.Embedding(movies_title_vocabulary.vocab_size(),64)])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)))

In [15]:
model = MovieLensModel(users_model,movie_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(rating.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 17s 539ms/step - factorized_top_k/top_1_categorical_accuracy: 6.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0052 - factorized_top_k/top_50_categorical_accuracy: 0.0453 - factorized_top_k/top_100_categorical_accuracy: 0.1019 - loss: 33077.2982 - regularization_loss: 0.0000e+00 - total_loss: 33077.2982
Epoch 2/3
25/25 [==============================] - 14s 548ms/step - factorized_top_k/top_1_categorical_accuracy: 1.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0048 - factorized_top_k/top_10_categorical_accuracy: 0.0147 - factorized_top_k/top_50_categorical_accuracy: 0.1045 - factorized_top_k/top_100_categorical_accuracy: 0.2094 - loss: 31015.7200 - regularization_loss: 0.0000e+00 - total_loss: 31015.7200
Epoch 3/3
25/25 [==============================] - 14s 546ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categor

In [16]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
recommends.index_from_dataset(movies.batch(100).map(lambda title: (title, model.movie_model(title))))

id_ = input('Enter the user_id: ')
_, titles = recommends(np.array([str(id_)]))
print('Top recommendation for user',id_,titles[0, :3])

Enter the user_id: 7
Top recommendation for user 7 tf.Tensor(
[b'Fire on the Mountain (1996)' b'Reluctant Debutante, The (1958)'
 b'Great Race, The (1965)'], shape=(3,), dtype=string)


In [17]:
! wget http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2023-01-18 15:20:33--  http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211011981 (201M) [application/x-httpd-php]
Saving to: ‘YearPredictionMSD.txt.zip’

YearPredictionMSD.t 100%[===================>] 201.24M  19.1MB/s    in 12s     

2023-01-18 15:20:45 (17.4 MB/s) - ‘YearPredictionMSD.txt.zip’ saved [211011981/211011981]



In [18]:
! unzip '/content/YearPredictionMSD.txt.zip'

Archive:  /content/YearPredictionMSD.txt.zip
  inflating: YearPredictionMSD.txt   


In [19]:

with open('/content/YearPredictionMSD.txt', "r") as file1:
    FileContent = file1.read()
    print(FileContent)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

